In [5]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [39]:
df_weather=pd.read_csv('기상데이터 0510 index X.csv')
df_weather.head()

,관측일자,temperature,temp_max,temp_min,rainfall,humidity,조사지역,연,월,일
0,1988-01-01,-2.4,-0.8,-4.4,0.5,64.0,서울,1988,1,1
1,1988-01-01,-2.6,0.5,-6.2,1.9,73.0,관악산,1988,1,1
2,1988-01-01,-4.1,-0.7,-8.6,0.0,66.0,춘천,1988,1,1
3,1988-01-01,-3.1,-1.0,-8.6,0.4,79.0,철원,1988,1,1
4,1988-01-01,3.7,6.5,0.1,0.0,57.0,속초,1988,1,1


In [40]:
df_grain=pd.read_csv('작물데이터 0510 index X.csv')
df_grain.head()

,temperature,humidity,rainfall,temp_min,temp_max,label
0,20.879744,82.002744,202.935536,18,28,rice
1,21.770462,80.319644,226.655537,18,28,rice
2,23.004459,82.320763,263.964248,18,28,rice
3,26.491096,80.158363,242.864034,18,28,rice
4,20.130175,81.604873,262.717340,18,28,rice


In [8]:
# 예측값을 같이 MinMaxScaler 돌려서 예측 모델
def predict_futre_mmx(region, month, pred_year):
    pred_data = OLS_model_predict(region, month, pred_year)
    print(pred_data)
    tmp_data =[]
    for data in pred_data :
        tmp_data.append(data[-1])
    print(tmp_data)
    df_pred  = predict_crops_mmx(tmp_data)
    return df_pred

def predict_crops_mmx(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'humidity', 'label']]
    xgb = XGBClassifier()  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    grain_data= grain_data.append({'temperature' :  data[0], 'temp_max' : data[1], 'temp_min' : data[2], 'rainfall' : data[3]*30, 'humidity' : data[4] } , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-1], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    print(grain_mmx[-1])
    df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-1])), columns=labelencoder.classes_).T
    df_pred.columns = ['확률']
    df_pred = df_pred.sort_values(by='확률',ascending=False)
    return_pred = df_pred.head(3)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -1]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([value, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [6]:
# 예측값을 같이 StandardScaler 돌려서 예측 모델
def predict_futre_std(region, month, pred_year):
    pred_data = OLS_model_predict(region, month, pred_year)
    print(pred_data)
    tmp_data =[]
    for data in pred_data :
        tmp_data.append(data[-1])
    print(tmp_data)
    df_pred  = predict_crops_scaler(tmp_data)
    return df_pred

def predict_crops_std(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'humidity', 'label']]
    xgb = XGBClassifier()  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = StandardScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    grain_data= grain_data.append({'temperature' :  data[0], 'temp_max' : data[1], 'temp_min' : data[2], 'rainfall' : data[3]*30, 'humidity' : data[4] } , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-1], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    print(grain_mmx[-1])
    df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-1])), columns=labelencoder.classes_).T
    df_pred.columns = ['확률']
    df_pred = df_pred.sort_values(by='확률',ascending=False)
    return_pred = df_pred.head(3)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -1]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([value, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [4]:
# MinMaxScaler 값에서 min / max 지표 뺀거

def predict_futre_x_mmx(region, month, pred_year):
    pred_data = OLS_model_predict(region, month, pred_year)
    print(pred_data)
    tmp_data =[]
    for i, data in enumerate(pred_data) :
        if i in [0, 3, 4] :
            tmp_data.append(data[-1])
    df_pred  = predict_crops_x_mmx(tmp_data)
    return df_pred

def predict_crops_x_mmx(data):
    tmp_grain = df_grain[['temperature', 'rainfall', 'humidity', 'label']]
    xgb = XGBClassifier()  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    grain_data=grain_data.append({'temperature' :  data[0], 'rainfall' : data[1]*30, 'humidity' : data[2] } , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-1], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    print(grain_mmx[-1])
    df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-1])), columns=labelencoder.classes_).T
    df_pred.columns = ['확률']
    df_pred = df_pred.sort_values(by='확률',ascending=False)
    return_pred = df_pred.head(3)
    return return_pred


def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -1]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([value, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list





In [3]:
# StandardScaler + min / max 지표 뺀거

def predict_futre_x_std(region, month, pred_year):
    pred_data = OLS_model_predict(region, month, pred_year)
    print(pred_data)
    tmp_data =[]
    for i, data in enumerate(pred_data) :
        if i in [0, 3, 4] :
            tmp_data.append(data[-1])
    df_pred  = predict_crops_x_std(tmp_data)
    return df_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -1]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([value, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list


def predict_crops_x_std(data):
    tmp_grain = df_grain[['temperature', 'rainfall', 'humidity', 'label']]
    xgb = XGBClassifier()  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = StandardScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    grain_data=grain_data.append({'temperature' :  data[0], 'rainfall' : data[1]*30, 'humidity' : data[2] } , ignore_index=True)

    grain_std = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_std[:-1])
    # display(grain_std[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_std[:-1], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    print(grain_std[-1])
    df_pred = pd.DataFrame(xgb.predict_proba(list(grain_std[-1])), columns=labelencoder.classes_).T
    df_pred.columns = ['확률']
    df_pred = df_pred.sort_values(by='확률',ascending=False)
    return_pred = df_pred.head(3)
    return return_pred




In [11]:
# predict_futre_mmx, predict_futre_std, predict_futre_x_mmx, predict_futre_x_std

In [43]:
def value_matrix(region, month, year):
     display(predict_futre_mmx(region, month, year)), 
     display(predict_futre_std(region, month, year)), 
     display(predict_futre_x_mmx(region, month, year)), 
     display(predict_futre_x_std(region, month, year))

In [45]:
value_matrix('서울', 6, 2022)

[['temperature', 22.80263004569429], ['temp_max', 32.66101643862828], ['temp_min', 14.510903067907272], ['rainfall', 4.682851271821746], ['humidity', 67.1027178427563]]
[22.80263004569429, 32.66101643862828, 14.510903067907272, 4.682851271821746, 67.1027178427563]
[0.4010625  0.69420902 0.43231378 0.43209904 0.61645256]


,확률
pigeonpeas,0.394745
coffee,0.042974
coconut,0.041548


[['temperature', 22.80263004569429], ['temp_max', 32.66101643862828], ['temp_min', 14.510903067907272], ['rainfall', 4.682851271821746], ['humidity', 67.1027178427563]]
[22.80263004569429, 32.66101643862828, 14.510903067907272, 4.682851271821746, 67.1027178427563]
[0.4010625  0.69420902 0.43231378 0.43209904 0.61645256]


,확률
pigeonpeas,0.394745
coffee,0.042974
coconut,0.041548


[['temperature', 22.80263004569429], ['temp_max', 32.66101643862828], ['temp_min', 14.510903067907272], ['rainfall', 4.682851271821746], ['humidity', 67.1027178427563]]
[0.4010625  0.43209904 0.61645256]


,확률
pigeonpeas,0.592883
coffee,0.231827
maize,0.094844


[['temperature', 22.80263004569429], ['temp_max', 32.66101643862828], ['temp_min', 14.510903067907272], ['rainfall', 4.682851271821746], ['humidity', 67.1027178427563]]
[-0.5555996   0.6735654  -0.19668794]


,확률
pigeonpeas,0.592883
coffee,0.231827
maize,0.094844


In [46]:
value_matrix('서귀포', 6, 2022)

[['temperature', 22.018025363961687], ['temp_max', 28.96616323037101], ['temp_min', 15.473696279163166], ['rainfall', 9.064290028600686], ['humidity', 81.68495739967017]]
[22.018025363961687, 28.96616323037101, 15.473696279163166, 9.064290028600686, 81.68495739967017]
[0.37854862 0.5402568  0.4760771  0.9043236  0.78655973]


,확률
rice,0.996480
cotton,0.000428
jute,0.000268


[['temperature', 22.018025363961687], ['temp_max', 28.96616323037101], ['temp_min', 15.473696279163166], ['rainfall', 9.064290028600686], ['humidity', 81.68495739967017]]
[22.018025363961687, 28.96616323037101, 15.473696279163166, 9.064290028600686, 81.68495739967017]
[0.37854862 0.5402568  0.4760771  0.9043236  0.78655973]


,확률
rice,0.996480
cotton,0.000428
jute,0.000268


[['temperature', 22.018025363961687], ['temp_max', 28.96616323037101], ['temp_min', 15.473696279163166], ['rainfall', 9.064290028600686], ['humidity', 81.68495739967017]]
[0.37854862 0.9043236  0.78655973]


,확률
rice,0.994690
jute,0.001737
cotton,0.000676


[['temperature', 22.018025363961687], ['temp_max', 28.96616323037101], ['temp_min', 15.473696279163166], ['rainfall', 9.064290028600686], ['humidity', 81.68495739967017]]
[-0.71050249  3.05879519  0.4582635 ]


,확률
rice,0.994690
jute,0.001737
cotton,0.000676
